In [1]:
import warnings 
warnings.filterwarnings("ignore")

In [2]:
%run ./analysis_utils/log_retriever.py

In [3]:
baseline_log_retriever = LogRetreiver(model="baseline", test_path="../../data/test/merged_test_data.csv")
_, baseline_pred_dfs = baseline_log_retriever.get_logs()

chunk_log_retriever = LogRetreiver(model="chunk_combinations", test_path="../../data/test/merged_test_data.csv")
_, chunk_pred_dfs = chunk_log_retriever.get_logs()

text_summarizer_log_retriever = LogRetreiver(model="text_summarizer", test_path="../../data/test/merged_test_data.csv")
_, text_summarizer_pred_dfs = text_summarizer_log_retriever.get_logs()

5
5
5


In [4]:
def merge_with_test(pred_dfs):
    merged_dfs = []
    test_df = pd.read_csv("../../data/test/merged_test_data.csv")
    for pred_df in pred_dfs:
        merged_df = pd.merge(test_df, pred_df, on="pair_id")
        if "overall_x" in merged_df.columns:
            merged_df = merged_df.rename(columns={"overall_x": "overall"})
            merged_df = merged_df.drop(columns=["overall_y"])
        merged_dfs.append(merged_df)

    return merged_dfs

def get_avg_dist(merged_dfs, model="baseline"):
    avg_dists = []
    for merged_df in merged_dfs:
        if model != "chunk_combinations":
            merged_df["dist"] = abs(merged_df["overall"] - merged_df["pred"])
        else:
            merged_df["dist"] = abs(merged_df["overall"] - merged_df["logits"])
        avg_dist = merged_df["dist"].mean()
        avg_dists.append(avg_dist)

    avg_dist = sum(avg_dists) / len(avg_dists)
    return avg_dist

def run(pred_dfs, model="baseline"):
    merged_dfs = merge_with_test(pred_dfs)
    avg_dist = get_avg_dist(merged_dfs, model=model)

    return avg_dist

In [5]:
baseline_avg_dist = run(baseline_pred_dfs, model="baseline")
chunk_avg_dist = run(chunk_pred_dfs, model="chunk_combinations")
text_summarizer_avg_dist = run(text_summarizer_pred_dfs, model="text_summarizer")

print(f"Baseline avg dist: {baseline_avg_dist}")
print(f"Chunk avg dist: {chunk_avg_dist}")
print(f"Text summarizer avg dist: {text_summarizer_avg_dist}")

Baseline avg dist: 0.6945433523699905
Chunk avg dist: 0.7595411479349905
Text summarizer avg dist: 0.7389031005472665


In [6]:
baseline_pred_dfs[0].head()

,true,pred,id1,id2,pair_id
0,2.0,1.275662,1544695356,1543856243,1544695356_1543856243
1,3.0,3.662091,1558575944,1558260621,1558575944_1558260621
2,3.0,3.940144,1587083102,1584460230,1587083102_1584460230
3,2.0,3.705279,1626919601,1627431440,1626919601_1627431440
4,4.0,2.987852,1578970555,1582067061,1578970555_1582067061


#### Pearson Correlation Coefficient

In [13]:
def calc_pearson_corr(pred_dfs, model="baseline"):
    pearson_corrs = []
    for pred_df in pred_dfs:
        if model != "chunk_combinations":
            pearson_corr = pred_df["true"].corr(pred_df["pred"], method="pearson")
        else:
            pearson_corr = pred_df["overall"].corr(pred_df["logits"], method="pearson")
        pearson_corrs.append(pearson_corr)

    return sum(pearson_corrs) / len(pearson_corrs)

In [14]:
baseline_pearson_corr = calc_pearson_corr(baseline_pred_dfs)
print(f"Baseline Pearson correlation: {baseline_pearson_corr}")

text_summarizer_pearson_corr = calc_pearson_corr(text_summarizer_pred_dfs)
print(f"Text summarizer Pearson correlation: {text_summarizer_pearson_corr}")

chunk_combinations_pearson_corr = calc_pearson_corr(chunk_pred_dfs, model="chunk_combinations")
print(f"Chunk combinations Pearson correlation: {chunk_combinations_pearson_corr}")

Baseline Pearson correlation: 0.677053163091454
Text summarizer Pearson correlation: 0.6321327449326901
Chunk combinations Pearson correlation: 0.6251145664467239
